In [4]:
import mysql.connector
import pandas as pd 
import json
import numpy as np
import MySQLdb
import MySQLdb.cursors

In [5]:
db_thread = MySQLdb.connect("127.0.0.1", "root", "andhika", "usek_monev", cursorclass=MySQLdb.cursors.DictCursor)
cur_thread = db_thread.cursor()
print(cur_thread)

In [6]:
cur_thread.execute("SELECT * from paket p where p.status = 0 and p.nama like '%K13_Seni%'")
pakets = cur_thread.fetchall()
for paket in pakets:
    sql = "SELECT * FROM soal where paket_id='%s'" % (paket['id'])
    soal = pd.read_sql(sql, con=db_thread)
    soal = soal.set_index('id')
    sql = "select * from ujian_siswa where paket_id='%s' and (status=2)" % (paket['id'])
    cur_thread.execute(sql)
    ujian_siswas = cur_thread.fetchall()
    print(len(ujian_siswas))
    for ujian_siswa in ujian_siswas:
        print("ujian siswa id: ", ujian_siswa['id'])
        soals = json.loads(ujian_siswa["random_soal"])
        jawaban_posisi = json.loads(ujian_siswa["random_jawaban"])
        jawabans = json.loads(ujian_siswa["jawaban_siswa"])
        for key, soal_id in enumerate(soals):
            siswa_id_benar = json.loads(soal.loc[soal_id, 'siswa_id_benar'])
            if (soal.loc[soal_id, 'tipe_soal'] == 'pilihan_ganda'):    
                intochar={'1':"a",'2':"b",'3':"c",'4':"d",'5':"e"}
                arr_random= {
                    "a" : intochar[jawaban_posisi[key][0]],
                    "b" : intochar[jawaban_posisi[key][1]],
                    "c" : intochar[jawaban_posisi[key][2]],
                    "d" : intochar[jawaban_posisi[key][3]],
                    "e" : intochar[jawaban_posisi[key][4]],
                }
                if(jawabans[key] ==" "):
                    jawaban_benar=0
                else:
                    if (arr_random[jawabans[key].lower()] == soal.loc[soal_id, 'kunci_jawaban'].lower()): 
                        jawaban_benar=1
                        siswa_id_benar.append(ujian_siswa['id'])
                        s = json.dumps(siswa_id_benar)
                        soal.loc[soal_id, 'siswa_id_benar'] = s
                    else : 
                        jawaban_benar=0
                        
                    if(arr_random[jawabans[key].lower()] == 'a'):
                        soal.loc[soal_id, 'jawaban_a'] += 1 
                    elif(arr_random[jawabans[key].lower()] == 'b'):
                        soal.loc[soal_id, 'jawaban_b'] += 1
                    elif(arr_random[jawabans[key].lower()] == 'c'): 
                        soal.loc[soal_id, 'jawaban_c'] += 1
                    elif(arr_random[jawabans[key].lower()] == 'd'):
                        soal.loc[soal_id, 'jawaban_d'] += 1
                    elif(arr_random[jawabans[key].lower()] == 'e'):
                        soal.loc[soal_id, 'jawaban_e'] += 1
                soal.loc[soal_id, 'jumlah_benar_siswa'] += jawaban_benar                
                soal.loc[soal_id, 'jumlah_siswa'] += 1
    for index, row in soal.iterrows():
        sql = "update soal set jawaban_a=%d, jawaban_b=%d, jawaban_c=%d, jawaban_d=%d, jawaban_e=%d, siswa_id_benar='%s', jumlah_benar_siswa=%d, jumlah_siswa=%d where id='%s'" %(row['jawaban_a'], row['jawaban_b'], row['jawaban_c'], row['jawaban_d'], row['jawaban_e'], row['siswa_id_benar'], row['jumlah_benar_siswa'], row['jumlah_siswa'], index)
        cur_thread.execute(sql)
    sql = "update ujian_siswa set status=3 where paket_id='%s' and (status=2)" % (paket['id'])
    cur_thread.execute(sql)
    sql = "update paket set status=1 where id='%s'" % (paket['id'])
    cur_thread.execute(sql)
    db_thread.commit()
    print("~DONE~ paket:", paket['nama'])

20086
ujian siswa id:  00068ff0-6276-11ea-9737-5f0053ad2fd9
ujian siswa id:  00070a20-6276-11ea-a896-fd4a3da0072a
ujian siswa id:  000b3300-6276-11ea-97ac-0d65f7866229
ujian siswa id:  000e2010-6277-11ea-8dcb-8de3901ce6d6
ujian siswa id:  0011f770-6276-11ea-b55c-db94881dd301
ujian siswa id:  00149370-6279-11ea-bbcc-37443ab40c75
ujian siswa id:  001c06e0-6276-11ea-8f33-21bd1de89430
ujian siswa id:  001f4090-6275-11ea-a178-c5e5d06cbc30
ujian siswa id:  002f3090-6276-11ea-b377-453a017042c8
ujian siswa id:  002fc8d0-6276-11ea-87fa-3db1ae23b8b9
ujian siswa id:  003391a0-6276-11ea-a326-f58bc85b3bc5
ujian siswa id:  00345ed0-6276-11ea-a36f-c775e6bc56bd
ujian siswa id:  00373fd0-6275-11ea-804a-4da6ad26df0a
ujian siswa id:  003893c0-6276-11ea-b808-7f88f2f4b67c
ujian siswa id:  00496100-6278-11ea-b76a-c104d14f6fc8
ujian siswa id:  004a08a0-6276-11ea-b5ad-f73a7d64d6e1
ujian siswa id:  004a3ba0-6276-11ea-a5f8-a93cf1545da2
ujian siswa id:  004aec50-6276-11ea-b943-9911503251d9
ujian siswa id:  004d7

ujian siswa id:  02578d20-6276-11ea-b935-f5ff07dec46f
ujian siswa id:  0258aad0-6276-11ea-8d5d-d7e383542a91
ujian siswa id:  025993e0-6276-11ea-bad0-13f8d2cfb7f4
ujian siswa id:  0262d2d0-6276-11ea-96df-7d270469b372
ujian siswa id:  027230d0-6276-11ea-b3d0-6100dea6fceb
ujian siswa id:  0273e9c0-6276-11ea-b38f-593f2b0074b2
ujian siswa id:  02761a80-627a-11ea-afd7-755aabdc026d
ujian siswa id:  0287bf60-6278-11ea-93a2-499918bd5dec
ujian siswa id:  028e2af0-6278-11ea-8be4-571163ee60d8
ujian siswa id:  0297bd80-6276-11ea-ae93-b55d2b9c0a7b
ujian siswa id:  029b3510-627c-11ea-b287-35268c755d8d
ujian siswa id:  029bf120-6277-11ea-a4c0-7ddc1c1596c7
ujian siswa id:  029dfd50-6276-11ea-9dc1-6d689b759b93
ujian siswa id:  02a07570-6275-11ea-aa2f-2f423a6ea581
ujian siswa id:  02a1d0f0-6276-11ea-b41c-698254e91a92
ujian siswa id:  02abf8d0-6276-11ea-911d-6f25021dbea2
ujian siswa id:  02b13240-6276-11ea-8cdb-3ff151a263f0
ujian siswa id:  02b33560-6276-11ea-97b3-9feefd7715bc
ujian siswa id:  02bd58a0-62

ujian siswa id:  04c07420-6276-11ea-b447-b50320758d1a
ujian siswa id:  04c9a100-6276-11ea-a8af-5db2041883ca
ujian siswa id:  04ca0540-6277-11ea-836c-59792b324231
ujian siswa id:  04cceb00-6276-11ea-8af6-01c9cc397286
ujian siswa id:  04cdf470-6277-11ea-a732-af8ab0b38196
ujian siswa id:  04d88bb0-6276-11ea-9341-37d4d1e0491f
ujian siswa id:  04db7cc0-6275-11ea-a4bc-6f74a6ca7836
ujian siswa id:  04dda8b0-627c-11ea-a0d8-b56a118568c2
ujian siswa id:  04e1be40-6276-11ea-84e5-5f91bcd5f946
ujian siswa id:  04e57d30-6276-11ea-8304-db66fa730a7b
ujian siswa id:  04e58260-6278-11ea-9e44-85a7b00eba20
ujian siswa id:  04f45cf0-6276-11ea-8edd-cd88919b20aa
ujian siswa id:  04f8ee30-6276-11ea-b31e-f3c38ebf8c60
ujian siswa id:  04ff7f60-6276-11ea-a230-d3166c4e30d4
ujian siswa id:  050672d0-6276-11ea-9e4e-db03ce2f57bd
ujian siswa id:  050bcd50-6275-11ea-9606-37ec8b212f76
ujian siswa id:  051384c0-6276-11ea-8b3d-afc7442ab912
ujian siswa id:  0514be30-6276-11ea-a755-891dc2a7676c
ujian siswa id:  0515d0f0-62

ujian siswa id:  071620d0-6277-11ea-8516-05a8047fb694
ujian siswa id:  071bf1b0-6279-11ea-a089-f7198f556c94
ujian siswa id:  0724dbe0-6277-11ea-bfbf-9be054481c35
ujian siswa id:  07300840-6276-11ea-9a05-911669118c9a
ujian siswa id:  07339b20-6278-11ea-9430-c979540d2dfa
ujian siswa id:  0739bc50-6276-11ea-bb2b-e39f6dd30f42
ujian siswa id:  073dc000-6276-11ea-a7d6-e7683429c3c7
ujian siswa id:  07427720-6275-11ea-a7a3-ddfec57dfd0b
ujian siswa id:  07444b20-6276-11ea-9626-759603747626
ujian siswa id:  07479aa0-6276-11ea-ac0e-a95ee3cc766f
ujian siswa id:  074829a0-6278-11ea-94e7-2d3fa20a4424
ujian siswa id:  07494c00-6275-11ea-8112-0f960e005b1b
ujian siswa id:  07503610-6278-11ea-a9f2-fb359fedef09
ujian siswa id:  07528930-627a-11ea-9866-616888a98b7d
ujian siswa id:  07550020-627e-11ea-9922-6356385c8922
ujian siswa id:  075884b0-627e-11ea-b2a3-59b92df30f42
ujian siswa id:  075a1aa0-6275-11ea-826d-150ec6bc3a40
ujian siswa id:  07718cb0-6276-11ea-a5e6-55574fe98d8e
ujian siswa id:  0774fc70-62

ujian siswa id:  095fc0f0-6276-11ea-97c9-095bd416414f
ujian siswa id:  0965fab0-6275-11ea-a499-dd948c42cfee
ujian siswa id:  09675640-627b-11ea-b9f2-6fd4214e073c
ujian siswa id:  096962e0-6277-11ea-88cc-8f675c428d75
ujian siswa id:  096b7d80-6277-11ea-a43f-799040842915
ujian siswa id:  097106f0-6276-11ea-8946-87e7b9f3f89f
ujian siswa id:  0978b340-6275-11ea-b244-717eb1e74054
ujian siswa id:  097faf00-6275-11ea-be5b-d3d3887632d6
ujian siswa id:  09860690-6276-11ea-a877-a348bfcbf8c3
ujian siswa id:  09867290-6276-11ea-b435-17f4674d0075
ujian siswa id:  098a6920-6275-11ea-9df8-2f344c5680fd
ujian siswa id:  098fc950-6276-11ea-b531-27f42fbc1346
ujian siswa id:  0996e8b0-6277-11ea-ad6d-a95e9941e4e5
ujian siswa id:  09972ca0-6275-11ea-9b33-cf1df36fc02d
ujian siswa id:  09990b40-6275-11ea-867b-5dd27cf7aa72
ujian siswa id:  099da4c0-6281-11ea-9e66-c1eb7a54839b
ujian siswa id:  09a7a7b0-627e-11ea-8c26-e1ad9e67a255
ujian siswa id:  09aa6930-6276-11ea-9a25-fbf4dd3bfffa
ujian siswa id:  09ab0a20-62

...

ujian siswa id:  7b3f7680-629f-11ea-b93f-b9c78292747c
ujian siswa id:  7b507a60-629f-11ea-9fad-c70665d3aaee
ujian siswa id:  7b6651a0-629f-11ea-acf3-eb40fba222d3
ujian siswa id:  7b6a46e0-629f-11ea-978b-8f80abe01638
ujian siswa id:  7b74aab0-629f-11ea-8cf1-93f0a2c00eba
ujian siswa id:  7b776b00-629f-11ea-b17f-17246f9a8774
ujian siswa id:  7b7e6700-629f-11ea-9760-2d747673ae58
ujian siswa id:  7b8aed50-62a1-11ea-abeb-5fe6fae408b3
ujian siswa id:  7b936e90-62a0-11ea-aac2-a35dd32f6aba
ujian siswa id:  7b989b80-629f-11ea-9d23-b1c344d1965b
ujian siswa id:  7b9eea00-62a0-11ea-b2ac-659656b85668
ujian siswa id:  7ba44df0-629f-11ea-ab20-d1cb67a585d5
ujian siswa id:  7ba45590-62a0-11ea-88cb-b143f5339e78
ujian siswa id:  7ba470e0-62a0-11ea-890e-4310d8d8370a
ujian siswa id:  7ba6c8a0-629f-11ea-ae13-1948261d842a
ujian siswa id:  7baea550-629f-11ea-a8e0-dde3e0542352
ujian siswa id:  7bbf0b00-629f-11ea-b495-a38df30d2578
ujian siswa id:  7bbfe670-62a0-11ea-b851-99d0c2fcf10d
ujian siswa id:  7bc6e890-62

ujian siswa id:  7f218670-62a0-11ea-836d-a7032f0a2db3
ujian siswa id:  7f2879e0-629f-11ea-8d3f-4f1578771fef
ujian siswa id:  7f2a3a40-629f-11ea-9cc0-5d6a78edd697
ujian siswa id:  7f2c88b0-629f-11ea-8f54-11103acde86b
ujian siswa id:  7f2f5340-629f-11ea-9381-cf1fa1c3aaa4
ujian siswa id:  7f2fa680-629f-11ea-a249-a53963e009b2
ujian siswa id:  7f435c20-62a0-11ea-8e7a-d35f02aa2357
ujian siswa id:  7f452cd0-629f-11ea-a402-e5032c81cbda
ujian siswa id:  7f468da0-62a0-11ea-b215-01de6002cdb2
ujian siswa id:  7f482da0-629f-11ea-816e-65a5431d8311
ujian siswa id:  7f4e6580-629f-11ea-8e53-4324be044fc6
ujian siswa id:  7f578370-629f-11ea-952d-efcf892d8c90
ujian siswa id:  7f594b10-629f-11ea-8b2b-c91cc0e5abba
ujian siswa id:  7f5b6e10-629f-11ea-a916-7d5ea1d97b52
ujian siswa id:  7f6104d0-62a2-11ea-ba3f-27ab5564ce17
ujian siswa id:  7f65c450-629f-11ea-95d7-b1843a90497d
ujian siswa id:  7f6db7a0-629f-11ea-8229-edca188d26e0
ujian siswa id:  7f777ff0-62a0-11ea-94a6-ef94bbc50c5e
ujian siswa id:  7f788a70-62

ujian siswa id:  82cd8ab0-62a0-11ea-a7b8-0bf24247fe0d
ujian siswa id:  82d00030-62a0-11ea-a67a-6b077850d269
ujian siswa id:  82db00a0-629f-11ea-9f65-a36679daa1cc
ujian siswa id:  82e6eda0-629f-11ea-8ae8-6795fa610c8f
ujian siswa id:  82e97c70-62a0-11ea-a7b0-332f94b7ff12
ujian siswa id:  8300b480-629f-11ea-8dbe-e508487d6c86
ujian siswa id:  8301b560-62a0-11ea-802d-33e22792096b
ujian siswa id:  83184400-62a0-11ea-aed6-1106a981c6ad
ujian siswa id:  831884c0-629f-11ea-8561-a378d75167fc
ujian siswa id:  831c2620-62a0-11ea-b130-9f883f0484d9
ujian siswa id:  83217a00-62a1-11ea-acec-4dff43148ad1
ujian siswa id:  832f88e0-629f-11ea-adb0-bb22e6e6942a
ujian siswa id:  8338cf30-629f-11ea-afe3-7d92c7f2beaa
ujian siswa id:  8368b190-629f-11ea-a073-71db17b7fc40
ujian siswa id:  836a0de0-629f-11ea-8a3d-c5a9c11fce29
ujian siswa id:  836b00a0-62a0-11ea-bf0f-b9e3c5fec828
ujian siswa id:  83734310-629f-11ea-b9c8-47d45d158890
ujian siswa id:  8379f870-62a1-11ea-a0f0-9dec955d5f14
ujian siswa id:  837e21b0-62

ujian siswa id:  87890c40-62a0-11ea-9915-4bb0eee332a1
ujian siswa id:  878bdae0-62a0-11ea-958c-394f8d7d5be4
ujian siswa id:  878f1d30-629f-11ea-ad51-bb77516a26ac
ujian siswa id:  8791dba0-629f-11ea-a72f-ddea24763b8d
ujian siswa id:  879dbba0-62a0-11ea-b454-c11ee8cefcc0


KeyboardInterrupt: 